In [1]:
import numpy as np
import pandas as pd
import sys; sys.path.append("/home/hugs/workstation/dmt")
from dmt.model import interface, adapter, Adapter, Interface, AIBase

SyntaxError: invalid syntax (__init__.py, line 131)

# DMT Nexus
DMT Nexus is a service to validate your models.


In [3]:
from enum import Enum

class EntityType(Enum):
    DATA = "data"
    ANALYSIS = "analysis"
    
class Data:
    """Data with some meta-data to track."""
    def __init__(self,
        phenomenon,
        data,
        provenance):
        """
        Arguments
        -----------
        phenomenon: Phenomenon measured by this data. 
        data: a pandas data-frame containing values for this data.
        provenance: a dictionary telling where did this data come from?"""
        self._entity_type = EntityType.DATA
        self.phenomenon = phenomenon
        self.data = data
        self.provenance = provenance
        
    @property
    def as_dict(self):
        """this data instance as a dict"""
        return dict(
            phenomenon=self.phenomenon,
            provenance=self.provenance,
            data=self.data)
    
class DMTNexus:
    """DMT Nexus is a service to validate your models."""
    def __init__(self):
        self._data = {}
        self._analyses = {}
        
    def __upload_data(self, data, phenomenon=None):
        """Upload data"""
        if not phenomenon and not hasattr(data, "phenomenon"):
            raise ValueError(
                "No phenomenon provided for the data to be uploaded.")
        phenomenon=\
            phenomenon if phenomenon\
            else getattr(data, "phenomenon", None)
        
        if phenomenon not in self._data:
            self._data[phenomenon] = []
        self._data[phenomenon].append(data)
        return len(self._data)
    
    def __upload_analysis(self, analysis, phenomenon=None):
        """Upload analysis"""
        if not phenomenon and not hasattr(analysis, "phenomenon"):
            raise ValueError(
                "No phenomenon provided for the analysis to be uploaded.")
        phenomenon=\
            phenomenon if phenomenon\
            else getattr(analysis, "phenomenon", None)
        
        if phenomenon not in self._analyses:
            self._analyses[phenomenon] = []
        self._analyses[phenomenon].append(analysis)
        return len(self._analyses)
            
    def upload(self, entity_type, entity, phenomenon=None):
        """Upload an entity
            Arguments
            ----------
            entity_type: EntityType
            entity: an instance of entity_type.
            
            Return
            -----------
            an identifier to track the uploaded data."""
        
        if entity_type == EntityType.DATA:
            return\
                self.__upload_data(
                    entity,
                    phenomenon=phenomenon)
        if entity_type == EntityType.ANALYSIS:
            return self.__upload_analysis(
                    entity,
                    phenomenon=phenomenon)
        raise ValueError(
            "Uploading of entity type {} is not yet supported.".format(entity_type))
        
    def get_data(self, phenomenon, identifier=None):
        """Get data by phenomenon"""
        available = self._data.get(phenomenon, [])
        try:
            return available[identifier - 1]
        except:
            print("Data {} for phenomenon {} not found"\
                  .format(identifier, phenomenon))
        return None
        
    def get_analyses(self, phenomenon, identifier=None):
        """Get analyses by phenomenon"""
        available = self._analyses.get(phenomenon, [])
        try:
            return available[identifier - 1]
        except:
            print("Analysis {} for phenomenon {} not found"\
                  .format(identifier, phenomenon))
        return None
        
        
dmt_nexus = DMTNexus()

In [4]:
dmt_nexus.upload(
    EntityType.DATA,
    Data(
        phenomenon = "cell_density",
        data       =  pd.DataFrame({
                        "mean": [0.8, 12., 20., 24., 10., 2.],
                        "std": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]},
                        index=pd.Index(
                            ["L1", "L2", "L3", "L4", "L5", "L6"],
                            name="layer")),
        provenance = {"uri": "this notebook",
                      "citation": "mock data"}))
    

1

In [5]:
class Verdict(Enum):
    PASS = 1
    INCONCLUSIVE = 0
    FAIL = -1
    
class CellDensityValidation(AIBase):
    """Validate cell densities"""
    def __init__(self,
            reference_data,
            pvalue_threshold = 0.05):
        """
        Arguments
        ----------
        pvalue_threshold: Float"""
        self.pvalue_threshold = pvalue_threshold
        self.phenomenon = "cell_density"
        self.reference_data = reference_data
        
    class AdapterInterface(Interface):
        """All methods listed here must be implemented 
        by an adapter of this interface."""
        
        def get_cell_density(self,
                circuit_model):
            """Get volume density of (neuronal) cells in a circuit.
            This method must be defined for the model adapter class that will
            adapt a circuit model to the requirements of this analysis.
            
            Arguments
            ---------------
            circuit_model: circuit model to be validated.
            
            Return
            ---------------
            A pandas.DataFrame containing the mean and standard deviation in 
            columns 'mean' and 'std' respectively, 
            and indexed by 'layer'"""
            pass
        
    def get_verdict(self,
            model_measurement):
        """Test the model data"""
        from dmt.vtk.statistics import FischersPooler
        from scipy.special import erf
        reference_data=\
            self.reference_data.data
        delta_mean=\
            np.abs(model_measurement["mean"] - reference_data["mean"])
        stdev=\
            np.sqrt(
                model_measurement["std"] ** 2 + reference_data["std"] ** 2)
        z_score=\
            delta_mean / stdev
        pvalue=\
            FischersPooler.eval(1. - erf(z_score))
        if np.isnan(pvalue):
            return{
                "status": Verdict.INCONCLUSIVE,
                "pvalue": pvalue}
        return{
            "status": Verdict.PASS if pvalue > self.pvalue_threshold else Verdict.FAIL,
            "pvalue": pvalue}
        
    def __call__(self,
        model,
        *args, **kwargs):
        """Call Me"""
        model_measurement=\
            self.adapter.get_cell_density(model)
        report=\
            dict(
                reference=self.reference_data.as_dict,
                model=model_measurement,
                verdict=self.get_verdict(model_measurement))
        return report


NameError: name 'AIBase' is not defined

In [6]:
class DumbCircuitModel:
    """The dumbest possible model for a brain circuit."""
    density = {layer: {"mean": i, "std": 0} for i, layer in enumerate(["L1", "L2", "L3", "L4", "L5", "L6"])}
    pass

@interface.implementation(CellDensityValidation.AdapterInterface)
@adapter.adapter(DumbestCircuitModel)
class DumbCircuitAdapter:
    """An adapter for the dumbest possible model for a brain circuit"""
    pass

NameError: name 'interface' is not defined

In [7]:
@interface.implementation(CellDensityValidation.AdapterInterface)
@adapter.adapter(DumbCircuitModel)
class DumbCircuitAdapter:
    """An adapter for the dumbest possible model for a brain circuit"""
    def get_cell_density(self, circuit_model):
        """return made up values."""
        layers = ["L1", "L2", "L3", "L4", "L5", "L6"]
        return\
            pd.DataFrame(
                {"mean": [circuit_model.density[l]["mean"] for l in layers],
                  "std": [circuit_model.density[l]["mean"] for l in layers]},
                index=pd.Index(layers, name="layer"))


NameError: name 'interface' is not defined

In [8]:
cell_density_validation=\
    CellDensityValidation(
            dmt_nexus.get_data("cell_density", identifier=1))

cell_density_validation.adapter = DumbCircuitAdapter()

NameError: name 'CellDensityValidation' is not defined

In [97]:
report = cell_density_validation(DumbCircuitModel())
print(report["verdict"])

{'pvalue': 0.0, 'status': <Verdict.FAIL: -1>}


In [98]:
dmt_nexus.upload(EntityType.ANALYSIS, report, phenomenon="cell_density")

1

In [99]:
class DumberCircuitModel:
    """it is even dumber"""
    density = {l: {'mean': 1, 'std': 0} for l in range(1, 7)}
    pass

@interface.implementation(CellDensityValidation.AdapterInterface)
@adapter.adapter(DumbCircuitModel)
@adapter.adapter(DumberCircuitModel)
class DumbAndDumberCircuitAdapter:
    """an adapter that works for dumb and dumber circuits"""
    def __init__(self, specialization):
        self.specialization = specialization

    def get_cell_density(self, circuit_model):
        layers = ["L1", "L2", "L3", "L4", "L5", "L6"]
        model_layer_labels = self.specialization.layer_labels(layers)
        return pd.DataFrame(
            {"mean": [circuit_model.density[l]["mean"] for l in model_layer_labels],
             "std": [circuit_model.density[l]["std"] for l in model_layer_labels]},  
            index=pd.Index(layers, name="layer"))
    
    
class DumbCircuitSpec:
    """specialization for dumb circuit"""
    def layer_labels(self, layers):
        return layers
    pass

class DumberCircuitSpec:
    """specialization for dumber circuit"""
    def layer_labels(self, layers):
        return [int(layer[1]) for layer in layers]

WARNING!!! AdapterInterface implmentation DumbAndDumberCircuitAdapter should attribute its author


In [81]:
cell_density_validation=\
    CellDensityValidation(
            dmt_nexus.get_data("cell_density", identifier=1))

cell_density_validation.adapter = DumbAndDumberCircuitAdapter(DumbCircuitSpec())
dumb_report = cell_density_validation(DumbCircuitModel())
cell_density_validation.adapter = DumbAndDumberCircuitAdapter(DumberCircuitSpec())
dumber_report = cell_density_validation(DumberCircuitModel())

In [82]:
print(dumb_report, dumber_report)

{'model':    mean  std
0     0    0
1     1    0
2     2    0
3     3    0
4     4    0
5     5    0, 'verdict': {'pvalue': nan, 'status': <Verdict.INCONCLUSIVE: 0>}, 'reference': {'provenance': {'citation': 'mock data', 'uri': 'this notebook'}, 'phenomenon': 'cell_density', 'data':        mean  std
layer           
L1      0.8  0.1
L2     12.0  0.1
L3     20.0  0.1
L4     24.0  0.1
L5     10.0  0.1
L6      2.0  0.1}} {'model':    mean  std
0     1    0
1     1    0
2     1    0
3     1    0
4     1    0
5     1    0, 'verdict': {'pvalue': nan, 'status': <Verdict.INCONCLUSIVE: 0>}, 'reference': {'provenance': {'citation': 'mock data', 'uri': 'this notebook'}, 'phenomenon': 'cell_density', 'data':        mean  std
layer           
L1      0.8  0.1
L2     12.0  0.1
L3     20.0  0.1
L4     24.0  0.1
L5     10.0  0.1
L6      2.0  0.1}}
